In [ ]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config

import dask
import io
import re
import logging
import s3fs

from astropy.io import fits
from dask.distributed import Client
from os import listdir
from os.path import isfile, join
from re import search

In [ ]:
from dask_gateway import Gateway, GatewayCluster
gateway = Gateway()
options = gateway.cluster_options()

# We're setting some defaults here just for grins... 
# I like the pangeo/base-notebook image for the workers since it has almost every library you'd need on a worker
# In our environment, without setting these, the widget will default to the same image that the notebook itself is running, 
# as well as 2 cores and 4GB memory per worker

options.image = 'public.ecr.aws/q3h7b4o8/heliocloud/helio-daskhub-mltf:2025.01.29'
options.worker_cores = 4
options.worker_memory = 7
options.profile='gpu-xlarge'

# This calls the widget
options  

In [ ]:
cluster = gateway.new_cluster(options)
client = cluster.get_client()
n_workers = 1
cluster.adapt(minimum=1, maximum=n_workers)
cluster

In [ ]:
# create client, show url we can go to to monitor progress
client = Client(cluster)
client

In [ ]:
%%time

import dask.array as da
a = da.random.normal(size=(30000,30000), chunks=(1000, 1000))
data = a.mean().compute()
print(data)
type(a)

In [ ]:
import numpy as np

array_a = np.random.rand(4000,6000).astype(np.float32)
array_b = np.random.rand(6000,4000).astype(np.float32)

data = da.matmul(array_a, array_b)

In [ ]:
import pickle
# let us save this
with open('pvc_test2.pickle','wb') as fout:
    pickle.dump(data, fout)

In [ ]:
cluster.shutdown()

In [ ]:
from dask.distributed import Client

client = Client(cluster)
client

In [ ]:
%%time

import dask.array as da
a = da.random.normal(size=(30000,30000), chunks=(1000, 1000))
out = a.mean().compute()
print(out)
